# Load Competition Dataset

Competition dataset located in "/kaggle/input"; This path defined by Kaggle to access the competition file. We will list two files from this path as input files.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path=os.path.join(dirname, filename)
        if 'train' in path:
            __training_path=path
        elif 'test' in path:
            __test_path=path

## Check training and test path

In [ ]:
#loaded files
print(f'Training path:{__training_path}\nTest path:{__test_path}')

In [ ]:
# Kaggle Environment Prepration
#update kaggle env
import sys
#you may update the environment that allow you to run the whole code
#!{sys.executable} -m pip install --upgrade scikit-learn=="0.24.2" 
#record this information if you need to run the Kernel internally
import sklearn; sklearn.show_versions() 

# Input Dataset

In [ ]:
def __load__data(__training_path, __test_path, concat=False):
	"""load data as input dataset
	params: __training_path: the training path of input dataset
	params: __test_path: the path of test dataset
	params: if it is True, then it will concatinate the training and test dataset as output
	returns: generate final loaded dataset as dataset, input and test
	"""
	# LOAD DATA
	import pandas as pd
	__train_dataset = pd.read_csv(__training_path, delimiter=',')
	__test_dataset = pd.read_csv(__test_path, delimiter=',')
	if not concat:
	    __dataset = __train_dataset.copy()
	else:
	    __dataset = pd.concat([__train_dataset, __test_dataset], axis=0
	        ).reset_index(drop=True)
	return __dataset, __train_dataset, __test_dataset
__dataset, __train_dataset, __test_dataset = __load__data(__training_path, __test_path, concat=True)
__dataset

# Generate Submission file
We have to maintain the following columns in submission.csv.
then, we can drop that column(s) from original dataset because it is unique and not useful for training a model.
In some cases, an ID may carry useful information such as student ID where it may consist of admission year and other related info.

In [ ]:
submission_columns = ['id']

In [ ]:
submission=pd.DataFrame(__test_dataset[submission_columns].copy())

In [ ]:
# DISCARD IRRELEVANT COLUMNS
__dataset.drop(['id'], axis=1, inplace=True)

## Drop Target Column
We need to drop target column from the training dataset.

Now let's drops target columns of <b>loss</b> from dataset.

### Set Target Column
The target column in the value which we need to predict.
Therefore, we need to detach the target columns in prediction.
Please note that if we don't drop this fields, it will generate a model with high accuracy on training and worst accuracy on test (because the value in test dataset is Null).

In [ ]:
target_column_pred = "loss"

In [ ]:
# DETATCH TARGET
__feature = __dataset.drop(['loss'], axis=1)
__target =__dataset['loss']

## Split Train/Test
We have to separate train and test before start training a model

In [ ]:
# TRAIN TEST SPLIT
__num_of_training_instances = __train_dataset.shape[0]
__feature_train = __feature.iloc[:__num_of_training_instances,:]
__feature_test = __feature.iloc[__num_of_training_instances:,:]
__target_train = __target.iloc[:__num_of_training_instances]
__target_test = __target.iloc[__num_of_training_instances:]
__feature_train

# Training Model and Prediction
First, we will train a model, then predict test values based on the trained model.

In [ ]:
# MODEL
from lightgbm import LGBMRegressor
__lgbmregressor=LGBMRegressor()
__lgbmregressor.fit(__feature_train, __target_train)
__y_pred = __lgbmregressor.predict(__feature_test)

In [ ]:
# Load target columns for prediction and generate submission file for the competition.
submission[target_column_pred]=__y_pred
submission

In [ ]:
# Generate a submission CSV file; we have to save the file as a CSV and avoid adding dataframe index into it.
submission.to_csv('submission.csv', index=False)

In [ ]:
# Now, let's review submission file.
submission